This file is a notebook to debug analyze the graphs produced by Swigg_ext.

In [1]:
from collections import defaultdict
import argparse

import numpy as np
import pandas as pd

from collections import Counter
from Bio import SeqIO
import networkx as nx

import matplotlib.pyplot as plt

import sys
import os
sys.setrecursionlimit(500524)

### Functions to contract nodes
Rules for supernodes: 

During DFS, continuously add node to the current supernode if the current node is a one-in-one-out node (in degree <=1 and out degree <=1). Otherwise, create a new supernode. 

In [2]:
class DFSOBJ:
    supernode_dict = defaultdict()
    supernodes = [[]]
    snid_g = 0
    sn_adj = defaultdict(list)

    def DFS(self, source, graph):
        visited = defaultdict()
        self.sn_adj = defaultdict(list)
        self.supernodes = [[]]
        self.snid_g = 0
        self.supernode_dict = defaultdict()
        return self.DFS_sub(source, visited, graph, False)

    def DFS_sub(self, curr, visited, graph, split):
        visited[curr] = 1
        # determine if a split is needed
        if split or graph.in_degree(curr)>1 or graph.out_degree(curr)>1:
            self.supernodes.append([curr])
            self.sn_adj[self.snid_g].append(self.snid_g+1)
            self.snid_g+=1
        else:
            self.supernodes[self.snid_g].append(curr)
        self.supernode_dict[curr] = self.snid_g
        if graph.out_degree(curr)>1:
            split = True
        else:
            split = False

        for n in graph.neighbors(curr):
            if n not in visited:
                self.DFS_sub(n, visited, graph, split)
            else:
                if self.supernode_dict[n] not in self.sn_adj[self.snid_g]:
                    self.sn_adj[self.snid_g].append(self.supernode_dict[n])

### Main Loop
If there are 1000 sequences in the fasta file, the script will randomly choose 100 sequences and repeat the process for 10 times.

If there are fewer than 1000 sequences, comment out the outer loop.

In [7]:
seq_list = []
k_lengths = [50]
fasta = ["../HIV2.fasta"] # could be multiple fasta files
outdir = "../HIV2/"  # make that your own directory (remember to mkdir before using it!) and preferably not in the git repo

outputname = os.path.basename(fasta[0]).split(".")[0]

for seqq in fasta:
    seq_list = seq_list + [(str(a.id),str(a.seq)) for a in list(SeqIO.parse(seqq, "fasta"))]

# random experiments
np.random.shuffle(seq_list)
    
# for seqn in range(0,1000,100):
# print("====== Seqn = "+ str(seqn))
# outputname = outputname+"_"+str(seqn)
seq_df = pd.DataFrame(seq_list)
#seq_df = pd.DataFrame(seq_list).head()
seq_df.columns = ["name","Sequence"]

name_dict = list(enumerate(seq_df["name"]))

for k_length in k_lengths:
    outputname = outputname + "_" + str(k_length)
    print("----- K-mer lengths = "+str(k_length))
    kmer_pos_dict = defaultdict(list)
    kmer_name_dict = defaultdict(list)
    kmer_pos_dict = defaultdict(list)
    kmer_dict = set()
    
    # go through each sequence and find all k-mers
    for i in range(len(seq_df)):
        name = i
        seq = seq_df.loc[i,"Sequence"]
        for j in range(len(seq)-k_length+1):
            kmer_dict.add(seq[j:j+k_length])
            
    # stores kmer-id pairs for lookup afterwards
    kmer_id_list = dict(zip(sorted(list(kmer_dict)), list(range(1,len(kmer_dict)+1))))
    id_kmer_list = dict(zip(list(range(1,len(kmer_dict)+1)),sorted(list(kmer_dict))))
    
    # go through the sequence again and store k-mer adjacencies
    edges = defaultdict(list)
    for i in range(len(seq_df)):
        name = i
        seq = seq_df.loc[i,"Sequence"]
        edges[0].append(kmer_id_list[seq[:k_length]])
        for j in range(len(seq)-k_length-1):
            edges[kmer_id_list[seq[j:j+k_length]]].append(kmer_id_list[seq[j+1:j+k_length+1]])
            kmer_name_dict[kmer_id_list[seq[j:j+k_length]]].append(name)   # stores the origin of the kmer (sample name)
            kmer_pos_dict[kmer_id_list[seq[j:j+k_length]]].append(j)       # stores the origin of the kmer (position in genome)
        kmer_name_dict[kmer_id_list[seq[len(seq)-k_length:]]].append(name)
        kmer_pos_dict[kmer_id_list[seq[len(seq)-k_length:]]].append(j)
    
    # create initial de bruijn graph
    G = nx.DiGraph()
    print("Number of nodes: ",len(kmer_id_list))
    edges_list = []
    for j in edges:
        for i in edges[j]:
            G.add_edge(j,i)
            edges_list.append((j,i))
    print("Number of edges: ",G.number_of_edges())

    print("Contracting nodes...")
    source = [i for i in G.in_degree() if i[1]==0] # find the source of the graph
    a = DFSOBJ()
    a.DFS(0, G)
    
    new_G = nx.DiGraph()
    for i in a.sn_adj:
        for j in a.sn_adj[i]:
            new_G.add_edge(i, j)
    print("Number of nodes: " + str(len(new_G.nodes)))
    print("Number of edges: " + str(len(new_G.edges)))
    nx.write_gexf(new_G, outdir+"compactSWIGG_"+outputname+".gexf")

----- K-mer lengths = 50
Number of nodes:  52794
Number of edges:  52910
Contracting nodes...
Number of nodes: 442
Number of edges: 558


FileNotFoundError: [Errno 2] No such file or directory: '../HIV2/compactSWIGG_HIV2_50.gexf'